<a href="https://colab.research.google.com/github/brendanpshea/programming_problem_solving/blob/main/Java_12_Pacman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's Build PacMan (with Swing)!

Welcome to one of the most exciting chapters in this textbook! Over the next several sections, you'll build a fully functional Pacman-style game from scratch. But we're not just going to hack together some code that works—we're going to build it *the right way*, using professional software architecture patterns that real game developers use.

In this first section, we'll introduce the **Model-View-Controller (MVC)** pattern, set up our project structure, and create the foundation of our application. By organizing our code properly from the start, we'll make it much easier to add features, fix bugs, and understand what each part of our code does. Let's get started!

## Understanding the MVC Pattern

The **Model-View-Controller (MVC)** pattern is a way of organizing code that separates different responsibilities into three distinct parts:

- **The Model** represents the game's data and rules—things like Pacman's position, the maze layout, ghost behavior, and the score. The Model doesn't know or care about how the game looks on screen; it just manages the "truth" of what's happening in the game.

- **The View** is responsible for displaying the game to the player—drawing Pacman, ghosts, the maze, and the score on the screen. The View asks the Model "what should I draw?" but never changes the game state itself.

- **The Controller** handles user input and tells the Model what to do—when the player presses an arrow key, the Controller tells Pacman to change direction. The Controller acts as the "bridge" between the player and the game logic.

This separation is incredibly powerful. If you want to change how the game looks, you only modify the View. If you want to add new game rules, you only modify the Model. This makes your code more **maintainable** (easier to update) and **testable** (easier to verify it works correctly).

## Setting Up the Project Structure

We'll organize our project using clear class names that reflect the MVC pattern. In BlueJ, all our classes will be in a single project folder, but we'll use naming conventions to keep things organized:

```
PacmanGame Project/
├── PacmanGame.java       (main class - sets up the window)
├── GameController.java   (controller - handles input & game loop)
├── GamePanel.java        (view - draws everything)
├── GameState.java        (model - overall game state)
├── Board.java            (model - the maze)
├── Pacman.java           (model - the player character)
├── Ghost.java            (model - enemy characters)
├── Position.java         (model - x, y coordinates)
└── Direction.java        (model - UP, DOWN, LEFT, RIGHT)
```

- The **Model classes** (Board, Pacman, Ghost, GameState, Position, Direction) contain all the game logic and data.
- The **View class** (GamePanel) contains all the drawing code.
- The **Controller class** (GameController) contains all the input-handling code.
- The **main class** (PacmanGame) creates the window and connects everything together.

Even though everything is in one folder, we'll maintain clean separation by following the MVC principle: Model classes never import Swing classes, View classes only draw what the Model tells them to, and the Controller coordinates between them. BlueJ's visual class diagram will help you see the relationships between these classes clearly!



## Creating the Main Application Window

Let's create our main class that sets up the game window. This will be a simple Swing application with a `JFrame`:

```java
// PacmanGame.java
import javax.swing.*;

public class PacmanGame {
    public static void main(String[] args) {
        SwingUtilities.invokeLater(() -> createAndShowGUI());
    }
    
    private static void createAndShowGUI() {
        var frame = new JFrame("Pacman Game");
        frame.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
        frame.setSize(600, 650);
        frame.setResizable(false);
        frame.setLocationRelativeTo(null);
        frame.setVisible(true);
    }
}
```

Notice the use of **`var`**, a modern Java keyword introduced in Java 10. Instead of writing `JFrame frame = new JFrame("Pacman Game")`, we can write `var frame = new JFrame("Pacman Game")`. Java is smart enough to figure out that `frame` is a `JFrame` based on what we're assigning to it. This is called **type inference**—the compiler infers (figures out) the type for us.

## Modern Java: The `var` Keyword

The `var` keyword makes code more readable when the type is obvious from the right-hand side:

```java
// Old way (still valid!)
JFrame frame = new JFrame("Pacman Game");
ArrayList<String> names = new ArrayList<String>();

// New way with var (more concise!)
var frame = new JFrame("Pacman Game");
var names = new ArrayList<String>();
```

**Important rules about `var`:**

- You can only use `var` for local variables (inside methods), not for fields or parameters.
- You must initialize the variable immediately—you can't write `var x;` without assigning a value.
- The compiler still enforces type safety—`var` is just shorthand, not "any type."

## Key Architecture Principles

| Principle | What It Means | Example in Our Game |
|-----------|---------------|---------------------|
| **Separation of Concerns** | Each part of the code has one clear job | The Model manages game rules; the View only draws |
| **Encapsulation** | Hide implementation details inside classes | Pacman's position is private; access it through methods |
| **Loose Coupling** | Parts don't depend too heavily on each other | The View doesn't create Pacman objects; it just draws them |
| **High Cohesion** | Related functionality stays together | All ghost behavior code is in the Ghost class |

## Looking Ahead

In the next section, we'll start building the Model by creating the game board and maze. You'll learn how to represent the maze as a 2D array and how to check for walls, pellets, and power pellets. The foundation we've laid here—a clean MVC structure and a modern Java mindset—will make everything that follows much more straightforward.

Remember: good architecture isn't about making code fancy; it's about making code that you (and others!) can understand and modify six months from now. Let's keep building!

In [1]:
# @title
%%html
<svg viewBox="0 0 700 500" xmlns="http://www.w3.org/2000/svg">
  <!-- Title -->
  <text x="350" y="30" font-family="Arial, sans-serif" font-size="24" font-weight="bold" text-anchor="middle" fill="#2c3e50">
    Model-View-Controller Pattern
  </text>

  <!-- Model Box -->
  <rect x="50" y="80" width="200" height="140" fill="#3498db" stroke="#2c3e50" stroke-width="3" rx="10"/>
  <text x="150" y="110" font-family="Arial, sans-serif" font-size="20" font-weight="bold" text-anchor="middle" fill="white">
    MODEL
  </text>
  <text x="150" y="140" font-family="Arial, sans-serif" font-size="14" text-anchor="middle" fill="white">
    Game Logic &amp; Data
  </text>
  <text x="150" y="165" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="white">
    • Pacman position
  </text>
  <text x="150" y="185" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="white">
    • Ghost behavior
  </text>
  <text x="150" y="205" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="white">
    • Score &amp; lives
  </text>

  <!-- View Box -->
  <rect x="450" y="80" width="200" height="140" fill="#2ecc71" stroke="#2c3e50" stroke-width="3" rx="10"/>
  <text x="550" y="110" font-family="Arial, sans-serif" font-size="20" font-weight="bold" text-anchor="middle" fill="white">
    VIEW
  </text>
  <text x="550" y="140" font-family="Arial, sans-serif" font-size="14" text-anchor="middle" fill="white">
    Display Graphics
  </text>
  <text x="550" y="165" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="white">
    • Draw maze
  </text>
  <text x="550" y="185" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="white">
    • Draw characters
  </text>
  <text x="550" y="205" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="white">
    • Display score
  </text>

  <!-- Controller Box -->
  <rect x="250" y="310" width="200" height="140" fill="#e74c3c" stroke="#2c3e50" stroke-width="3" rx="10"/>
  <text x="350" y="340" font-family="Arial, sans-serif" font-size="20" font-weight="bold" text-anchor="middle" fill="white">
    CONTROLLER
  </text>
  <text x="350" y="370" font-family="Arial, sans-serif" font-size="14" text-anchor="middle" fill="white">
    User Input Handler
  </text>
  <text x="350" y="395" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="white">
    • Arrow keys
  </text>
  <text x="350" y="415" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="white">
    • Start/pause
  </text>
  <text x="350" y="435" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="white">
    • Game loop
  </text>

  <!-- Arrow from Controller to Model -->
  <defs>
    <marker id="arrowhead" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#2c3e50"/>
    </marker>
  </defs>

  <path d="M 300 310 L 180 230" stroke="#2c3e50" stroke-width="3" fill="none" marker-end="url(#arrowhead)"/>
  <text x="230" y="260" font-family="Arial, sans-serif" font-size="13" fill="#2c3e50" font-weight="bold">
    Updates
  </text>

  <!-- Arrow from Model to View -->
  <path d="M 250 150 L 450 150" stroke="#2c3e50" stroke-width="3" fill="none" marker-end="url(#arrowhead)"/>
  <text x="330" y="140" font-family="Arial, sans-serif" font-size="13" fill="#2c3e50" font-weight="bold">
    Provides data
  </text>

  <!-- Arrow from View to Controller (feedback) -->
  <path d="M 500 220 L 400 310" stroke="#2c3e50" stroke-width="3" fill="none" marker-end="url(#arrowhead)"/>
  <text x="460" y="270" font-family="Arial, sans-serif" font-size="13" fill="#2c3e50" font-weight="bold">
    Requests refresh
  </text>

  <!-- User interaction -->
  <ellipse cx="350" cy="480" rx="60" ry="25" fill="#f39c12" stroke="#2c3e50" stroke-width="2"/>
  <text x="350" y="487" font-family="Arial, sans-serif" font-size="14" font-weight="bold" text-anchor="middle" fill="white">
    PLAYER
  </text>

  <path d="M 350 455 L 350 450" stroke="#2c3e50" stroke-width="3" fill="none" marker-end="url(#arrowhead)"/>
  <text x="310" y="467" font-family="Arial, sans-serif" font-size="12" fill="#2c3e50">
    Input
  </text>
</svg>

# Game Board & Maze: Representing the Game World

Now that we have our project structure set up, let's start building the Model—the heart of our game. The first and most fundamental piece of the Model is the game board itself: the maze where Pacman runs around eating pellets and avoiding ghosts.

In this section, you'll learn how to represent a complex game world using simple data structures. We'll use a 2D array to store the maze layout and an enum to represent different types of tiles. By the end of this section, you'll have a `Board` class that knows everything about the maze without caring at all about how it gets drawn on screen. This is the MVC pattern in action!

## The Maze as a 2D Array

A Pacman maze is essentially a grid. Each cell in the grid can contain different things: a wall, an empty space, a pellet, or a power pellet. The natural way to represent a grid in Java is with a **2D array**—an array of arrays.

Think of it like a spreadsheet or a checkerboard. The first index represents the row (y-coordinate), and the second index represents the column (x-coordinate):

```java
// A simple 3x3 grid
int[][] grid = new int[3][3];

// Access the cell at row 1, column 2
grid[1][2] = 5;
```

For our Pacman game, instead of storing integers, we'll store tile types using an enum.

## Creating the TileType Enum

An **enum** (short for "enumeration") is a special Java type that represents a fixed set of constants. It's perfect for representing the different types of tiles in our maze.

```java
// save as TileType.java
public enum TileType {
    WALL,
    EMPTY,
    PELLET,
    POWER_PELLET
}
```

Each value (WALL, EMPTY, etc.) is called an **enum constant**. Using an enum is much better than using integers or strings because:

- The compiler catches typos—you can't accidentally write `TileType.WLAL`.
- Your code is self-documenting—`TileType.WALL` is clearer than `1`.
- You can't use invalid values—the only valid TileTypes are the ones you defined.

## The Board Class

Now let's create the `Board` class that will hold our maze. Here's the complete implementation with detailed comments:

```java
// save as Board.java
public class Board {
    // Constants for board dimensions
    private static final int WIDTH = 19;
    private static final int HEIGHT = 21;
    
    // The maze layout - private to enforce encapsulation
    private TileType[][] tiles;
    
    /**
     * Constructor - initializes the board with a default maze layout
     */
    public Board() {
        tiles = new TileType[HEIGHT][WIDTH];
        initializeMaze();
    }
    
    /**
     * Sets up a simple maze layout
     * This is a simplified version - you'll expand this later
     */
    private void initializeMaze() {
        // Fill the entire board with pellets first
        for (int row = 0; row < HEIGHT; row++) {
            for (int col = 0; col < WIDTH; col++) {
                tiles[row][col] = TileType.PELLET;
            }
        }
        
        // Add walls around the border
        for (int col = 0; col < WIDTH; col++) {
            tiles[0][col] = TileType.WALL;              // Top wall
            tiles[HEIGHT - 1][col] = TileType.WALL;     // Bottom wall
        }
        for (int row = 0; row < HEIGHT; row++) {
            tiles[row][0] = TileType.WALL;              // Left wall
            tiles[row][WIDTH - 1] = TileType.WALL;      // Right wall
        }
        
        // Add power pellets in the four corners (inside the walls)
        tiles[1][1] = TileType.POWER_PELLET;
        tiles[1][WIDTH - 2] = TileType.POWER_PELLET;
        tiles[HEIGHT - 2][1] = TileType.POWER_PELLET;
        tiles[HEIGHT - 2][WIDTH - 2] = TileType.POWER_PELLET;
    }
    
    /**
     * Gets the tile type at a specific position
     * This is a "getter" method that provides controlled access
     *
     * @param row the row (y-coordinate)
     * @param col the column (x-coordinate)
     * @return the TileType at that position, or WALL if out of bounds
     */
    public TileType getTileAt(int row, int col) {
        // Bounds checking - return WALL if position is invalid
        if (row < 0 || row >= HEIGHT || col < 0 || col >= WIDTH) {
            return TileType.WALL;
        }
        return tiles[row][col];
    }
    
    /**
     * Sets the tile type at a specific position
     * Used when Pacman eats a pellet (changes it to EMPTY)
     *
     * @param row the row (y-coordinate)
     * @param col the column (x-coordinate)
     * @param type the new TileType for that position
     */
    public void setTileAt(int row, int col, TileType type) {
        if (row >= 0 && row < HEIGHT && col >= 0 && col < WIDTH) {
            tiles[row][col] = type;
        }
    }
    
    /**
     * Checks if a specific position contains a wall
     * This is useful for collision detection
     *
     * @param row the row to check
     * @param col the column to check
     * @return true if the tile is a wall, false otherwise
     */
    public boolean isWall(int row, int col) {
        return getTileAt(row, col) == TileType.WALL;
    }
    
    /**
     * Gets the width of the board
     * @return the number of columns
     */
    public int getWidth() {
        return WIDTH;
    }
    
    /**
     * Gets the height of the board
     * @return the number of rows
     */
    public int getHeight() {
        return HEIGHT;
    }
}
```



## Understanding Encapsulation

Notice that the `tiles` array is **private**. This is **encapsulation**—hiding the internal details of how the class works. Other parts of our program can't directly access `tiles`; they must use the public methods (`getTileAt`, `setTileAt`, etc.).

Why is this important? Several reasons:

- **Validation**: The `getTileAt` method checks if the position is valid before accessing the array. Without encapsulation, other code could crash the program with invalid indices.
- **Flexibility**: Later, we could change how we store the maze (maybe loading it from a file) without breaking other parts of the program.
- **Clarity**: The public methods clearly show what operations are allowed on the Board.

## Reminder: Getter and Setter Methods

**Getter methods** allow other classes to read private data in a controlled way. They typically start with `get` and return a value:

```java
public int getWidth() {
    return WIDTH;
}
```

**Setter methods** allow other classes to modify private data in a controlled way. They typically start with `set` and take a parameter:

```java
public void setTileAt(int row, int col, TileType type) {
    tiles[row][col] = type;
}
```

The key word is *controlled*. You decide what's allowed. For example, our Board doesn't have a `setWidth()` method because the width should never change after the Board is created.

## Testing Your Board Class

In BlueJ, you can test your Board class interactively:

1. Right-click on the Board class and select "new Board()".
2. Right-click on the red board object and select "getTileAt(int, int)".
3. Try different row and column values to see what tiles are where.
4. Try accessing position (0, 0)—it should return WALL.
5. Try accessing position (5, 5)—it should return PELLET.

This kind of interactive testing is one of BlueJ's best features!

## Looking Ahead

In the next section, we'll create helper classes for Position and Direction. These will make it much easier to move Pacman and the ghosts around the board without constantly juggling separate x and y coordinates. We'll also explore how modern Java features like records can make our code even cleaner.

The Board class you just created is the foundation of the entire game. Every other part of the Model will use this Board to know where walls are, where pellets are, and where characters can move. That's the power of good design—build one solid piece at a time!

# Position & Direction: Making Movement Easier with Helper Classes

Now that we have a game board, we need a way to keep track of where things are on that board. We could use separate `x` and `y` variables everywhere, but that gets messy fast—imagine passing four parameters to a method just to move from one position to another! Instead, we'll create two small but powerful helper classes that will make our code much cleaner and easier to understand.

In this section, you'll create a `Direction` enum to represent the four cardinal directions and a `Position` class to bundle x and y coordinates together. These simple classes will be used throughout the rest of the project, demonstrating how well-designed small pieces can make everything else easier to build.

## The Direction Enum

Movement in Pacman is simple: up, down, left, or right. An enum is perfect for representing these four possibilities:

```java
// Direction.java
public enum Direction {
    UP,
    DOWN,
    LEFT,
    RIGHT
}
```

That's it! But we can make this enum even more useful by adding methods. Enums in Java can have methods just like classes can:

```java
// Direction.java
public enum Direction {
    UP,
    DOWN,
    LEFT,
    RIGHT;
    
    /**
     * Gets the change in row (dy) for this direction
     * @return -1 for UP, +1 for DOWN, 0 for LEFT/RIGHT
     */
    public int getDeltaRow() {
        return switch (this) {
            case UP -> -1;
            case DOWN -> 1;
            case LEFT, RIGHT -> 0;
        };
    }
    
    /**
     * Gets the change in column (dx) for this direction
     * @return -1 for LEFT, +1 for RIGHT, 0 for UP/DOWN
     */
    public int getDeltaCol() {
        return switch (this) {
            case LEFT -> -1;
            case RIGHT -> 1;
            case UP, DOWN -> 0;
        };
    }
    
    /**
     * Gets the opposite direction
     * @return the direction 180 degrees from this one
     */
    public Direction getOpposite() {
        return switch (this) {
            case UP -> DOWN;
            case DOWN -> UP;
            case LEFT -> RIGHT;
            case RIGHT -> LEFT;
        };
    }
}
```

## Modern Java: Switch Expressions

Notice the **switch expressions** in the code above. This is a modern Java feature (introduced in Java 14). Here's how it works:

- Switch expressions **return a value** directly, so you don't need separate return statements.
- You use **arrow syntax** (`->`) instead of colons and break statements.
- You can **combine multiple cases** with commas: `case LEFT, RIGHT -> 0`.
- The compiler **ensures you handle all cases**, catching bugs at compile-time.



## The Position Class

A position is simply an x and y coordinate, but bundling them together makes our code much cleaner:

```java
public class Position {
    private final int row;
    private final int col;
    
    /**
     * Creates a new position
     * @param row the row (y-coordinate)
     * @param col the column (x-coordinate)
     */
    public Position(int row, int col) {
        this.row = row;
        this.col = col;
    }
    
    /**
     * Gets the row
     * @return the row coordinate
     */
    public int getRow() {
        return row;
    }
    
    /**
     * Gets the column
     * @return the column coordinate
     */
    public int getCol() {
        return col;
    }
    
    /**
     * Creates a new Position by moving in a direction
     * This does NOT modify the current position (immutability!)
     *
     * @param direction the direction to move
     * @return a NEW Position object
     */
    public Position move(Direction direction) {
        return new Position(
            row + direction.getDeltaRow(),
            col + direction.getDeltaCol()
        );
    }
    
    /**
     * Checks if two positions are equal
     * @param obj the object to compare
     * @return true if both positions have the same row and col
     */
    @Override
    public boolean equals(Object obj) {
        if (this == obj) return true;
        if (!(obj instanceof Position)) return false;
        Position other = (Position) obj;
        return this.row == other.row && this.col == other.col;
    }
    
    /**
     * Generates a hash code for this position
     * Needed for using Position in HashMaps or HashSets
     * @return a hash code based on row and col
     */
    @Override
    public int hashCode() {
        return 31 * row + col;
    }
    
    /**
     * Returns a string representation of this position
     * @return a string like "Position(5, 10)"
     */
    @Override
    public String toString() {
        return "Position(" + row + ", " + col + ")";
    }
}
```

## Understanding Immutability

Notice that the `row` and `col` fields are **final**. This means once a Position is created, it can never be changed. The Position class is **immutable**.

The `move()` method doesn't modify the current position—it creates and returns a *new* Position object. This is a key principle of good design:

```java
Position start = new Position(5, 5);
Position next = start.move(Direction.RIGHT);

System.out.println(start);  // Still Position(5, 5)
System.out.println(next);   // Now Position(5, 6)
```

Why is immutability useful?

- **Prevents bugs**: Once you create a Position, you know it will never change unexpectedly.
- **Thread safety**: Multiple parts of your program can share the same Position without worrying about conflicts.
- **Easier reasoning**: You don't have to track when and where a Position might be modified.

| **Mutable Approach** | **Immutable Approach** |
|----------------------|------------------------|
| Fields can be changed after creation | Fields are final and cannot be changed |
| Methods like `move()` modify the object | Methods like `move()` return new objects |
| `position.move(Direction.RIGHT)` changes position | `position = position.move(Direction.RIGHT)` creates new position |
| Risk of unexpected side effects | No side effects—original stays unchanged |
| Example: `ArrayList`, `StringBuilder` | Example: `String`, `Integer`, our `Position` |

## Reminder: The `this` Keyword

In the Position constructor, you see `this.row = row`. The **`this` keyword** refers to the current object—the object whose method or constructor is being executed.

When a parameter has the same name as a field, you need `this` to distinguish them:

```java
public Position(int row, int col) {
    this.row = row;  // this.row is the field, row is the parameter
    this.col = col;  // this.col is the field, col is the parameter
}
```

You can also use `this` to call other methods on the same object:

```java
public void someMethod() {
    this.move(Direction.UP);  // Calls move() on this object
}
```

## Testing Position and Direction

Try creating and testing these classes in BlueJ:

1. Create a Direction object by right-clicking the Direction enum.
2. Call `getDeltaRow()` and `getDeltaCol()` on it—do the values make sense?
3. Create a Position object at (10, 15).
4. Call `move(Direction.RIGHT)` on it—you should get a new Position at (10, 16).
5. Verify that the original Position is still (10, 15)—demonstrating immutability!

## Looking Ahead

With Position and Direction complete, we have all the building blocks we need to create our game characters. In the next section, we'll build the Pacman class that uses these helper classes to move around the board, track animation state, and respond to player input. The clean abstractions we've created will make the Pacman class much simpler than it would be otherwise!

In [2]:
# @title
%%html
<svg viewBox="0 0 700 600" xmlns="http://www.w3.org/2000/svg">
  <!-- Title -->
  <text x="350" y="30" font-family="Arial, sans-serif" font-size="22" font-weight="bold" text-anchor="middle" fill="#2c3e50">
    Direction Movement on the Grid
  </text>

  <!-- Grid -->
  <g id="grid">
    <!-- Draw grid lines -->
    <line x1="100" y1="100" x2="600" y2="100" stroke="#ddd" stroke-width="1"/>
    <line x1="100" y1="200" x2="600" y2="200" stroke="#ddd" stroke-width="1"/>
    <line x1="100" y1="300" x2="600" y2="300" stroke="#ddd" stroke-width="1"/>
    <line x1="100" y1="400" x2="600" y2="400" stroke="#ddd" stroke-width="1"/>
    <line x1="100" y1="500" x2="600" y2="500" stroke="#ddd" stroke-width="1"/>

    <line x1="100" y1="100" x2="100" y2="500" stroke="#ddd" stroke-width="1"/>
    <line x1="200" y1="100" x2="200" y2="500" stroke="#ddd" stroke-width="1"/>
    <line x1="300" y1="100" x2="300" y2="500" stroke="#ddd" stroke-width="1"/>
    <line x1="400" y1="100" x2="400" y2="500" stroke="#ddd" stroke-width="1"/>
    <line x1="500" y1="100" x2="500" y2="500" stroke="#ddd" stroke-width="1"/>
    <line x1="600" y1="100" x2="600" y2="500" stroke="#ddd" stroke-width="1"/>

    <!-- Center position -->
    <circle cx="300" cy="300" r="30" fill="#f39c12" stroke="#2c3e50" stroke-width="3"/>
    <text x="300" y="310" font-family="Arial, sans-serif" font-size="16" font-weight="bold" text-anchor="middle" fill="white">
      START
    </text>
    <text x="300" y="340" font-family="Courier New, monospace" font-size="12" text-anchor="middle" fill="#2c3e50" font-weight="bold">
      (2, 2)
    </text>
  </g>

  <!-- Arrow markers -->
  <defs>
    <marker id="arrowRed" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#e74c3c"/>
    </marker>
    <marker id="arrowBlue" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#3498db"/>
    </marker>
    <marker id="arrowGreen" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#27ae60"/>
    </marker>
    <marker id="arrowPurple" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#9b59b6"/>
    </marker>
  </defs>

  <!-- UP arrow -->
  <line x1="300" y1="270" x2="300" y2="210" stroke="#e74c3c" stroke-width="4" marker-end="url(#arrowRed)"/>
  <circle cx="300" cy="200" r="15" fill="#e74c3c" opacity="0.3"/>
  <text x="300" y="180" font-family="Arial, sans-serif" font-size="14" font-weight="bold" text-anchor="middle" fill="#e74c3c">
    UP
  </text>
  <text x="300" y="165" font-family="Courier New, monospace" font-size="11" text-anchor="middle" fill="#e74c3c">
    (1, 2)
  </text>
  <text x="320" y="240" font-family="Courier New, monospace" font-size="11" fill="#e74c3c" font-weight="bold">
    Δrow = -1
  </text>
  <text x="320" y="253" font-family="Courier New, monospace" font-size="11" fill="#e74c3c" font-weight="bold">
    Δcol = 0
  </text>

  <!-- DOWN arrow -->
  <line x1="300" y1="330" x2="300" y2="390" stroke="#3498db" stroke-width="4" marker-end="url(#arrowBlue)"/>
  <circle cx="300" cy="400" r="15" fill="#3498db" opacity="0.3"/>
  <text x="300" y="430" font-family="Arial, sans-serif" font-size="14" font-weight="bold" text-anchor="middle" fill="#3498db">
    DOWN
  </text>
  <text x="300" y="445" font-family="Courier New, monospace" font-size="11" text-anchor="middle" fill="#3498db">
    (3, 2)
  </text>
  <text x="320" y="360" font-family="Courier New, monospace" font-size="11" fill="#3498db" font-weight="bold">
    Δrow = +1
  </text>
  <text x="320" y="373" font-family="Courier New, monospace" font-size="11" fill="#3498db" font-weight="bold">
    Δcol = 0
  </text>

  <!-- LEFT arrow -->
  <line x1="270" y1="300" x2="210" y2="300" stroke="#27ae60" stroke-width="4" marker-end="url(#arrowGreen)"/>
  <circle cx="200" cy="300" r="15" fill="#27ae60" opacity="0.3"/>
  <text x="200" y="280" font-family="Arial, sans-serif" font-size="14" font-weight="bold" text-anchor="middle" fill="#27ae60">
    LEFT
  </text>
  <text x="200" y="265" font-family="Courier New, monospace" font-size="11" text-anchor="middle" fill="#27ae60">
    (2, 1)
  </text>
  <text x="235" y="290" font-family="Courier New, monospace" font-size="11" text-anchor="middle" fill="#27ae60" font-weight="bold">
    Δrow = 0
  </text>
  <text x="235" y="303" font-family="Courier New, monospace" font-size="11" text-anchor="middle" fill="#27ae60" font-weight="bold">
    Δcol = -1
  </text>

  <!-- RIGHT arrow -->
  <line x1="330" y1="300" x2="390" y2="300" stroke="#9b59b6" stroke-width="4" marker-end="url(#arrowPurple)"/>
  <circle cx="400" cy="300" r="15" fill="#9b59b6" opacity="0.3"/>
  <text x="400" y="280" font-family="Arial, sans-serif" font-size="14" font-weight="bold" text-anchor="middle" fill="#9b59b6">
    RIGHT
  </text>
  <text x="400" y="265" font-family="Courier New, monospace" font-size="11" text-anchor="middle" fill="#9b59b6">
    (2, 3)
  </text>
  <text x="365" y="290" font-family="Courier New, monospace" font-size="11" text-anchor="middle" fill="#9b59b6" font-weight="bold">
    Δrow = 0
  </text>
  <text x="365" y="303" font-family="Courier New, monospace" font-size="11" text-anchor="middle" fill="#9b59b6" font-weight="bold">
    Δcol = +1
  </text>

  <!-- Coordinate labels -->
  <text x="80" y="105" font-family="Arial, sans-serif" font-size="12" text-anchor="end" fill="#666">row 0</text>
  <text x="80" y="205" font-family="Arial, sans-serif" font-size="12" text-anchor="end" fill="#666">row 1</text>
  <text x="80" y="305" font-family="Arial, sans-serif" font-size="12" text-anchor="end" fill="#666">row 2</text>
  <text x="80" y="405" font-family="Arial, sans-serif" font-size="12" text-anchor="end" fill="#666">row 3</text>
  <text x="80" y="505" font-family="Arial, sans-serif" font-size="12" text-anchor="end" fill="#666">row 4</text>

  <text x="100" y="530" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="#666">col 0</text>
  <text x="200" y="530" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="#666">col 1</text>
  <text x="300" y="530" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="#666">col 2</text>
  <text x="400" y="530" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="#666">col 3</text>
  <text x="500" y="530" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="#666">col 4</text>
  <text x="600" y="530" font-family="Arial, sans-serif" font-size="12" text-anchor="middle" fill="#666">col 5</text>

  <!-- Legend box -->
  <rect x="50" y="550" width="600" height="40" fill="#f8f9fa" stroke="#2c3e50" stroke-width="1" rx="5"/>
  <text x="350" y="575" font-family="Arial, sans-serif" font-size="13" text-anchor="middle" fill="#2c3e50">
    <tspan font-weight="bold">Remember:</tspan> In array notation [row][col], moving UP decreases row, moving RIGHT increases col
  </text>
</svg>

# Pacman Entity: Creating Our Hero

Now comes the exciting part: creating Pacman himself! The Pacman class will bring together everything we've built so far—the Board to know where walls are, the Position to track where Pacman is, and the Direction to know which way he's moving. This class will manage Pacman's location, movement, and animation state, all without knowing anything about how he'll be drawn on screen (that's the View's job!).

In this section, you'll see how a well-designed Model class uses other Model classes to create behavior. You'll also practice working with **instance variables**—the data that makes each object unique—and see how the `this` keyword helps distinguish between parameters and fields.

## Designing the Pacman Class

Before we write code, let's think about what Pacman needs to know about himself:

- **Position**: Where is Pacman on the board right now?
- **Current direction**: Which way is Pacman facing and moving?
- **Next direction**: Which way does the player want Pacman to turn (queued for when possible)?
- **Speed**: How fast does Pacman move (measured in pixels per frame)?
- **Animation state**: For drawing the "mouth" opening and closing (a number that increases over time).

These pieces of data are called **instance variables** or **fields**—they belong to each individual Pacman object and define its current state.

## The Complete Pacman Class

Here's the full implementation with detailed comments explaining each part:

```java
public class Pacman {
    // Instance variables - each Pacman object has its own copy of these
    private Position position;
    private Direction currentDirection;
    private Direction nextDirection;
    private double speed;
    private int animationFrame;
    
    // Constants
    private static final double DEFAULT_SPEED = 2.0;
    
    /**
     * Constructor - creates a new Pacman at the specified position
     *
     * @param startRow the starting row position
     * @param startCol the starting column position
     */
    public Pacman(int startRow, int startCol) {
        this.position = new Position(startRow, startCol);
        this.currentDirection = Direction.RIGHT;  // Start facing right
        this.nextDirection = Direction.RIGHT;
        this.speed = DEFAULT_SPEED;
        this.animationFrame = 0;
    }
    
    /**
     * Gets Pacman's current position
     * @return the current Position object
     */
    public Position getPosition() {
        return position;
    }
    
    /**
     * Gets Pacman's current direction
     * @return the Direction Pacman is facing
     */
    public Direction getCurrentDirection() {
        return currentDirection;
    }
    
    /**
     * Sets the next direction for Pacman to turn
     * The turn will happen when Pacman reaches a tile where turning is possible
     *
     * @param direction the direction the player wants to turn
     */
    public void setNextDirection(Direction direction) {
        this.nextDirection = direction;
    }
    
    /**
     * Gets the animation frame for drawing
     * @return a number that increases over time for animation
     */
    public int getAnimationFrame() {
        return animationFrame;
    }
    
    /**
     * Updates Pacman's position by moving in the current direction
     * This is called every game tick (frame)
     *
     * @param board the game board (to check for walls)
     */
    public void move(Board board) {
        // Try to turn to the next direction if possible
        Position nextPos = position.move(nextDirection);
        if (!board.isWall(nextPos.getRow(), nextPos.getCol())) {
            currentDirection = nextDirection;
        }
        
        // Move in the current direction if possible
        Position newPos = position.move(currentDirection);
        if (!board.isWall(newPos.getRow(), newPos.getCol())) {
            position = newPos;
        }
        
        // Update animation (cycles from 0 to 7, then repeats)
        animationFrame = (animationFrame + 1) % 8;
    }
    
    /**
     * Checks if Pacman can move in a specific direction
     * Useful for AI or testing
     *
     * @param board the game board
     * @param direction the direction to check
     * @return true if the move is valid (not into a wall)
     */
    public boolean canMove(Board board, Direction direction) {
        Position testPos = position.move(direction);
        return !board.isWall(testPos.getRow(), testPos.getCol());
    }
    
    /**
     * Resets Pacman to a new position (used when Pacman loses a life)
     *
     * @param row the row to reset to
     * @param col the column to reset to
     */
    public void reset(int row, int col) {
        this.position = new Position(row, col);
        this.currentDirection = Direction.RIGHT;
        this.nextDirection = Direction.RIGHT;
        this.animationFrame = 0;
    }
}
```



## Understanding Instance Variables

**Instance variables** (also called **fields**) are variables declared inside a class but outside any method. Each object of that class gets its own copy of these variables:

```java
Pacman player1 = new Pacman(10, 10);
Pacman player2 = new Pacman(15, 15);

// player1 and player2 have DIFFERENT positions
// They are separate objects with separate data
```

Instance variables are usually declared `private` to enforce encapsulation. Other classes can only access them through **getter methods** (like `getPosition()`) and **setter methods** (like `setNextDirection()`).

## The `this` Keyword Revisited

In the constructor, you see statements like `this.position = new Position(startRow, startCol)`. The **`this` keyword** refers to "the current object"—the specific Pacman object being constructed.

```java
public Pacman(int startRow, int startCol) {
    this.position = new Position(startRow, startCol);
    // "this.position" means "this Pacman's position field"
    
    this.currentDirection = Direction.RIGHT;
    // "this.currentDirection" means "this Pacman's currentDirection field"
}
```

You need `this` when:

- **Distinguishing parameters from fields**: When a parameter has the same name as a field.
- **Calling other methods on the same object**: `this.move(board)` calls the move method on the current Pacman.
- **Passing the current object to another method**: `someMethod(this)` passes the current Pacman as an argument.

In the `move()` method, we don't need `this` because there's no naming conflict, but we *could* write `this.position` for clarity:

```java
public void move(Board board) {
    // These are equivalent:
    position = newPos;           // Implicit "this"
    this.position = newPos;      // Explicit "this"
}
```

## How Movement Works

The `move()` method has two important steps:

1. **Try to turn**: Check if Pacman can move in the `nextDirection`. If so, update `currentDirection` to match. This allows responsive controls—the player can press a direction key early, and Pacman will turn as soon as possible.

2. **Move forward**: Try to move in the `currentDirection`. If there's a wall, Pacman just stays put.

Notice how we use the immutable Position class:

```java
Position newPos = position.move(currentDirection);
if (!board.isWall(newPos.getRow(), newPos.getCol())) {
    position = newPos;  // Replace old position with new position
}
```

We create a new Position, check if it's valid, and then replace the old position if appropriate. The old Position object is discarded (Java's garbage collector will clean it up automatically).

## Animation State

The `animationFrame` variable cycles from 0 to 7 repeatedly. Later, when we draw Pacman, we'll use this number to determine how wide open Pacman's mouth should be. A higher frame number means a more open mouth, creating the classic "chomping" animation.

```java
animationFrame = (animationFrame + 1) % 8;
```

The **modulo operator** (`%`) gives us the remainder after division. `(animationFrame + 1) % 8` counts: 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, ...

## Testing Your Pacman Class

In BlueJ, you can test Pacman interactively:

1. Create a Board object first (right-click Board → new Board()).
2. Create a Pacman object at position (10, 10).
3. Right-click the Pacman object and inspect it—you should see its position and direction.
4. Call `setNextDirection(Direction.LEFT)` to change direction.
5. Call `move(board1)` several times—watch Pacman's position change!
6. Try moving Pacman into a wall—he should stop at the wall instead of moving through it.

This hands-on testing helps you understand how objects maintain state and how methods modify that state.

## Looking Ahead

In the next section, we'll create the Ghost class. Ghosts are more complex than Pacman because they have different behavior modes (chase, scatter, frightened) and need AI to decide which direction to move. But the foundation we've built—Position, Direction, and the idea of an entity moving on a Board—will make ghosts much easier to implement than if we'd started from scratch!

The Pacman class demonstrates a crucial principle: **objects should know how to manage their own state**. Pacman knows where he is, which way he's facing, and how to move himself. This makes the rest of our code simpler because we can just call `pacman.move(board)` instead of managing all of Pacman's data manually.


# Ghost Entities: Creating the Enemies

What's Pacman without ghosts to chase him? In this section, we'll create the Ghost class—the adversaries that make the game challenging. Ghosts are more sophisticated than Pacman because they need artificial intelligence to decide where to move, and they have different behavior **modes** that change how they act during gameplay.

You'll see how to use enums to represent different states, how switch expressions make mode-based logic clean and readable, and how creating multiple Ghost instances with different "personalities" makes the game more interesting. By the end of this section, you'll have autonomous enemies that can chase Pacman, retreat to corners, and even run away when frightened!

## Ghost Behavior Modes

Real Pacman ghosts have three distinct behavior modes:

- **CHASE mode**: The ghost actively pursues Pacman, trying to catch him. This is the "aggressive" mode where the ghost is dangerous.
- **SCATTER mode**: The ghost retreats to its assigned corner of the maze. This gives players a brief respite and makes the game feel less relentless.
- **FRIGHTENED mode**: When Pacman eats a power pellet, ghosts become frightened and move randomly. Pacman can eat them during this time for bonus points!

We'll represent these modes with an enum:

```java
// Ghostmode.java
public enum GhostMode {
    CHASE,
    SCATTER,
    FRIGHTENED
}
```

## Ghost Personality: Target Corners

Each ghost has a "personality" defined by its target corner during SCATTER mode. In classic Pacman, the four ghosts scatter to the four corners of the maze, making them predictable but giving the player breathing room:

```java
// GhostPersonality.java
public enum GhostPersonality {
    BLINKY,  // Red ghost - targets top-right
    PINKY,   // Pink ghost - targets top-left
    INKY,    // Cyan ghost - targets bottom-right
    CLYDE    // Orange ghost - targets bottom-left
}
```

## The Complete Ghost Class

Here's the full Ghost implementation:

```java
// Ghost.java
public class Ghost {
    // Instance variables
    private Position position;
    private Direction currentDirection;
    private GhostMode mode;
    private GhostPersonality personality;
    private Position scatterTarget;  // Corner to retreat to in SCATTER mode
    
    // Constants
    private static final double CHASE_SPEED = 1.5;
    private static final double FRIGHTENED_SPEED = 0.8;
    
    /**
     * Constructor - creates a new Ghost
     *
     * @param startRow starting row position
     * @param startCol starting column position
     * @param personality the ghost's personality (determines scatter target)
     */
    public Ghost(int startRow, int startCol, GhostPersonality personality) {
        this.position = new Position(startRow, startCol);
        this.currentDirection = Direction.UP;  // Start facing up
        this.mode = GhostMode.SCATTER;  // Start in scatter mode
        this.personality = personality;
        this.scatterTarget = getScatterTargetForPersonality(personality);
    }
    
    /**
     * Gets the ghost's current position
     * @return the current Position object
     */
    public Position getPosition() {
        return position;
    }
    
    /**
     * Gets the ghost's current direction
     * @return the Direction the ghost is facing
     */
    public Direction getCurrentDirection() {
        return currentDirection;
    }
    
    /**
     * Gets the ghost's current mode
     * @return the current GhostMode
     */
    public GhostMode getMode() {
        return mode;
    }
    
    /**
     * Gets the ghost's personality
     * @return the GhostPersonality
     */
    public GhostPersonality getPersonality() {
        return personality;
    }
    
    /**
     * Sets the ghost's mode
     * @param newMode the new mode to enter
     */
    public void setMode(GhostMode newMode) {
        this.mode = newMode;
    }
    
    /**
     * Determines the scatter target corner based on personality
     * Uses a switch expression for clean, concise code
     *
     * @param personality the ghost's personality
     * @return the Position of the target corner
     */
    private Position getScatterTargetForPersonality(GhostPersonality personality) {
        return switch (personality) {
            case BLINKY -> new Position(0, 18);    // Top-right corner
            case PINKY -> new Position(0, 0);      // Top-left corner
            case INKY -> new Position(20, 18);     // Bottom-right corner
            case CLYDE -> new Position(20, 0);     // Bottom-left corner
        };
    }
    
    /**
     * Updates the ghost's position using AI to choose direction
     *
     * @param board the game board
     * @param pacmanPos Pacman's current position (for chase mode)
     */
    public void move(Board board, Position pacmanPos) {
        // Determine target based on current mode
        Position target = switch (mode) {
            case CHASE -> pacmanPos;           // Chase Pacman
            case SCATTER -> scatterTarget;     // Go to corner
            case FRIGHTENED -> null;           // Random movement (no target)
        };
        
        // Choose the best direction to move
        Direction newDirection;
        if (mode == GhostMode.FRIGHTENED) {
            newDirection = chooseRandomDirection(board);
        } else {
            newDirection = chooseBestDirection(board, target);
        }
        
        // Update direction if valid
        if (newDirection != null) {
            currentDirection = newDirection;
        }
        
        // Move in current direction
        Position newPos = position.move(currentDirection);
        if (!board.isWall(newPos.getRow(), newPos.getCol())) {
            position = newPos;
        }
    }
    
    /**
     * Chooses the best direction to move toward a target
     * Simple AI: pick the direction that reduces distance to target
     *
     * @param board the game board
     * @param target the target position to move toward
     * @return the best Direction, or null if no valid moves
     */
    private Direction chooseBestDirection(Board board, Position target) {
        Direction bestDir = null;
        double bestDistance = Double.MAX_VALUE;
        
        // Try all four directions
        for (Direction dir : Direction.values()) {
            // Don't reverse direction (ghosts don't go backwards)
            if (dir == currentDirection.getOpposite()) {
                continue;
            }
            
            Position testPos = position.move(dir);
            
            // Check if this direction is valid (not a wall)
            if (!board.isWall(testPos.getRow(), testPos.getCol())) {
                // Calculate distance to target
                double distance = calculateDistance(testPos, target);
                
                // Keep track of the best direction
                if (distance < bestDistance) {
                    bestDistance = distance;
                    bestDir = dir;
                }
            }
        }
        
        return bestDir;
    }
    
    /**
     * Chooses a random valid direction (for frightened mode)
     *
     * @param board the game board
     * @return a random valid Direction
     */
    private Direction chooseRandomDirection(Board board) {
        // Get all valid directions (not walls, not backwards)
        var validDirections = new java.util.ArrayList<Direction>();
        
        for (Direction dir : Direction.values()) {
            if (dir == currentDirection.getOpposite()) {
                continue;  // Don't reverse
            }
            
            Position testPos = position.move(dir);
            if (!board.isWall(testPos.getRow(), testPos.getCol())) {
                validDirections.add(dir);
            }
        }
        
        // Choose randomly from valid directions
        if (validDirections.isEmpty()) {
            return currentDirection;  // Keep going same way if stuck
        }
        
        int randomIndex = (int)(Math.random() * validDirections.size());
        return validDirections.get(randomIndex);
    }
    
    /**
     * Calculates the straight-line distance between two positions
     * Uses the Pythagorean theorem: distance = sqrt((x2-x1)² + (y2-y1)²)
     *
     * @param pos1 the first position
     * @param pos2 the second position
     * @return the distance between the positions
     */
    private double calculateDistance(Position pos1, Position pos2) {
        int deltaRow = pos1.getRow() - pos2.getRow();
        int deltaCol = pos1.getCol() - pos2.getCol();
        return Math.sqrt(deltaRow * deltaRow + deltaCol * deltaCol);
    }
    
    /**
     * Resets the ghost to a starting position
     *
     * @param row the row to reset to
     * @param col the column to reset to
     */
    public void reset(int row, int col) {
        this.position = new Position(row, col);
        this.currentDirection = Direction.UP;
        this.mode = GhostMode.SCATTER;
    }
}
```


## Simple Ghost AI

The `chooseBestDirection()` method implements a basic **pathfinding algorithm**. For each possible direction, it:

1. Checks if moving that way would hit a wall.
2. Calculates the straight-line distance to the target.
3. Picks the direction with the shortest distance.

This isn't perfect (ghosts can get stuck or take suboptimal paths), but it's simple and creates believable behavior. In a more advanced implementation, you'd use algorithms like A* (A-star) for optimal pathfinding.

## A Note on Inheritance

You might have noticed that Pacman and Ghost share some similarities—both have a position, a direction, and a `move()` method. In a larger project, you might create an `Entity` superclass that both extend:

```java
public abstract class Entity {
    protected Position position;
    protected Direction currentDirection;
    
    public abstract void move(Board board);
    public Position getPosition() { return position; }
    // ... shared methods
}

public class Pacman extends Entity { /* ... */ }
public class Ghost extends Entity { /* ... */ }
```

This is **inheritance**—where subclasses inherit fields and methods from a parent class. We're keeping things simple by not using inheritance here, but it's a natural extension if you continue developing the game!

## Creating Multiple Ghosts

In your game state, you'll create four ghosts with different personalities:

```java
Ghost blinky = new Ghost(9, 9, GhostPersonality.BLINKY);
Ghost pinky = new Ghost(9, 10, GhostPersonality.PINKY);
Ghost inky = new Ghost(10, 9, GhostPersonality.INKY);
Ghost clyde = new Ghost(10, 10, GhostPersonality.CLYDE);
```

Each ghost behaves slightly differently because they scatter to different corners, making the gameplay more varied and strategic.

## Looking Ahead

In the next section, we'll begin building the View—the part that draws everything on screen! We'll create a `GamePanel` class that extends `JPanel` and implements `paintComponent()` to render the maze, pellets, Pacman, and ghosts. You'll finally see your Model come to life visually!

The Ghost class demonstrates how **state-based behavior** works—the same ghost object acts differently depending on its mode. This pattern appears throughout game development and many other programming domains.